In [1]:
import torch
import torch.tensor as t
import torch.nn as nn
from random import randint

In [2]:
batch = 8
height = 50
width = 50

In [3]:
x = torch.rand(batch, 3, height, width)
x = nn.Softmax(dim=1)(x)
print(x.shape)

torch.Size([8, 3, 50, 50])


In [4]:
for i in range(10):
    b, h, w = randint(0, batch-1), randint(0, height-1), randint(0, width-1)
    print(x[b, :, h, w].sum().item())

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [5]:
x_new = x.permute(0, 2, 3, 1).contiguous().view(-1, 3)
# x_new = x_new
print(x_new.shape)

torch.Size([20000, 3])


In [6]:
n = x_new.shape[1]
for i in range(100):
    idx = randint(0, n-1)
    print(x_new[idx, :].sum().item())

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
